In [4]:
import ee
import geemap
import time

In [5]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
#for download files from araucaria update 05_Araucaria2gee

predios = r'D:\Raul Perez\scripts2023\Gee_Arauco\data\r20_2023\id_1.shp'
predios = geemap.shp_to_ee(predios)

In [7]:
prefire_start = '2023-09-29'
prefire_end = '2023-11-01'

# Now set the same parameters for AFTER the fire.
postfire_start = '2024-03-01'
#postfire_end = time.strftime('%Y-%m-%d', time.localtime(time.time()))
postfire_end = '2024-03-30'

bufferfirms = 1

# Define the cloud percentage threshold
cloud_percentage_threshold = 10


In [8]:
ImCol = 'COPERNICUS/S2_SR_HARMONIZED'
# SPPP = 'projects/gl-for-gis-dsr/assets/images/Limite_SPPP'
# firms = 'FIRMS'

In [9]:
imagery = ee.ImageCollection(ImCol)
# area = ee.FeatureCollection(SPPP)
# firms = ee.ImageCollection(firms)

In [100]:
# fires_firms = ee.ImageCollection(firms\
#         .select('T21')\
#         .filterDate(prefire_start,postfire_end)\
#         .filterBounds(area)) 
# fires_firmsmx = fires_firms.max().clip(area).subtract(273.15)
# fires_firmsrc = fires_firmsmx.gte(80).add(fires_firmsmx.gte(100)).add(fires_firmsmx.gte(150))
# fires_firmsvc= fires_firmsrc.reduceToVectors(**{
#   'geometry':area,
#   'crs': 'EPSG:32718',
#   'scale':500,
#   'geometryType':'polygon',
#   'eightConnected': False,
#   'bestEffort':False,
#   'maxPixels':9e10,
# }
#   )

# fires_firmsvc=ee.FeatureCollection(fires_firmsvc.toList(fires_firmsvc.size()))
# fires_firmsvc = fires_firmsvc.map(lambda f: f.buffer(bufferfirms))
# fires_firmsvc = fires_firmsvc.union()

In [10]:

def addNBR_s2h(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
    image = image.addBands(nbr)
    return image

prefireImCol = ee.ImageCollection(imagery \
    .filterDate(prefire_start, prefire_end) \
    .filterBounds(predios))\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloud_percentage_threshold)\
    .map(addNBR_s2h)

postfireImCol = ee.ImageCollection(imagery \
    .filterDate(postfire_start, postfire_end) \
    .filterBounds(predios))\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloud_percentage_threshold)\
    .map(addNBR_s2h)

# Function to mask clouds from the pixel quality band of Sentinel-2 SR data.
def maskS2sr(image):
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = ee.Number(2).pow(10).int()
  cirrusBitMask = ee.Number(2).pow(11).int()
  # Get the pixel QA band.
  qa = image.select('QA60')
  # All flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  # Return the masked image, scaled to TOA reflectance, without the QA bands.
  return image.updateMask(mask) \
      .copyProperties(image, ["system:time_start"])

prefire_CM_ImCol = prefireImCol.map(maskS2sr)
postfire_CM_ImCol = postfireImCol.map(maskS2sr)

pre_cm_mos = prefire_CM_ImCol.min().clip(predios)
post_cm_mos = postfire_CM_ImCol.mosaic().clip(predios)


preNBR = pre_cm_mos.select('nbr')
postNBR = post_cm_mos.select('nbr')

dNBR_withoutsmooth = preNBR.subtract(postNBR)
dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
    reducer=ee.Reducer.mean(),
    kernel=ee.Kernel.gaussian(radius=3),
)
#dNBR_unscaled=dNBR_unscaled.clip(r20)
dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
#dNBR_Seguro = dNBR_unscaled.gte(0.27)


# dNBR_unscaled = preNBR.subtract(postNBR)
# dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
# #dNBR_Seguro = dNBR_unscaled.gte(0.27)

viz_dNBR={
  'min':1,
  'max':7,#"#7a8737","#acbe4d","#0ae042",
  'palette':["#7a8737","#acbe4d","#0ae042","#fff70b","#ffaf38","#ff641b","#a41fd6"]
}

In [104]:
# vizfalsoColor = {
#   'bands': ['B8', 'B4', 'B3'],
#   'min': 700,
#   'max': 4000,
#   'gamma': [0.95, 1, 1]
# }
# Map.addLayer(post_cm_mos,vizfalsoColor,"Post - Fire FalsoColor")
# vizfalsoColor = {
#   'bands': ['B8', 'B4', 'B3'],
#   'min': 700,
#   'max': 4000,
#   'gamma': [0.95, 1, 1]
# }
# Map.addLayer(pre_cm_mos,vizfalsoColor,"Pre fire - FalsoColor")
# Map.layers

In [11]:
dNBR_vector= dNBR_reclass.reduceToVectors(**{
'geometry':predios,
'crs': 'EPSG:32718',
'scale':10,
'geometryType':'polygon',
'eightConnected': False,
'bestEffort':False,
'maxPixels':9e10,
'labelProperty': 'id_dano',
}
)

dissolved_polygon_fc = dissolve_by_attribute(dNBR_vector, 'id_dano')
    #.filterMetadata('label', 'greater_than', 3)


dNDBR_export=ee.FeatureCollection(dissolved_polygon_fc.toList(dNBR_vector.size()))
#Dissolve the simplified polygons and reduce file size

geemap.ee_export_vector_to_drive(
    dNDBR_export, description="dNBR_2023_postfire_10m_1", fileFormat='GeoJSON', folder="dNBR"
)

NameError: name 'dissolve_by_attribute' is not defined

In [12]:
predios = geemap.shp_to_ee(predios)
def addNBR_s2h(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename("nbr")
    image = image.addBands(nbr)
    return image

prefireImCol = ee.ImageCollection(imagery \
    .filterDate(prefire_start, prefire_end) \
    .filterBounds(predios))\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloud_percentage_threshold)\
    .map(addNBR_s2h)

postfireImCol = ee.ImageCollection(imagery \
    .filterDate(postfire_start, postfire_end) \
    .filterBounds(predios))\
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloud_percentage_threshold)\
    .map(addNBR_s2h)

# Function to mask clouds from the pixel quality band of Sentinel-2 SR data.
def maskS2sr(image):
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    # Get the pixel QA band.
    qa = image.select('QA60')
    # All flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    # Return the masked image, scaled to TOA reflectance, without the QA bands.
    return image.updateMask(mask) \
        .copyProperties(image, ["system:time_start"])


_getfullpathname: path should be string, bytes or os.PathLike, not FeatureCollection


In [121]:
for i in range(134,135):
    print(i)
    #for download files from araucaria update 05_Araucaria2gee

    predios = f'D:/Raul Perez/scripts2023/Gee_Arauco/data/r20_2023/id_{i}.shp'
    predios = geemap.shp_to_ee(predios)

    prefireImCol = ee.ImageCollection(imagery \
        .filterDate(prefire_start, prefire_end) \
        .filterBounds(predios))\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloud_percentage_threshold)\
        .map(addNBR_s2h)

    postfireImCol = ee.ImageCollection(imagery \
        .filterDate(postfire_start, postfire_end) \
        .filterBounds(predios))\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloud_percentage_threshold)\
        .map(addNBR_s2h)

    prefire_CM_ImCol = prefireImCol.map(maskS2sr)
    postfire_CM_ImCol = postfireImCol.map(maskS2sr)

    pre_cm_mos = prefire_CM_ImCol.min().clip(predios)
    post_cm_mos = postfire_CM_ImCol.mosaic().clip(predios)


    preNBR = pre_cm_mos.select('nbr')
    postNBR = post_cm_mos.select('nbr')

    dNBR_withoutsmooth = preNBR.subtract(postNBR)
    dNBR_unscaled = dNBR_withoutsmooth.reduceNeighborhood(
        reducer=ee.Reducer.mean(),
        kernel=ee.Kernel.gaussian(radius=3),
    )
    #dNBR_unscaled=dNBR_unscaled.clip(r20)
    dNBR_reclass = dNBR_unscaled.gte(-0.5).add(dNBR_unscaled.gte(-0.25)).add(dNBR_unscaled.gte(-0.1)).add(dNBR_unscaled.gte(0.1)).add(dNBR_unscaled.gte(0.27)).add(dNBR_unscaled.gte(0.44)).add(dNBR_unscaled.gte(0.66))
    #dNBR_Seguro = dNBR_unscaled.gte(0.27)

    dNBR_vector= dNBR_reclass.reduceToVectors(**{
    'geometry':predios,
    'crs': 'EPSG:32718',
    'scale':15,
    'geometryType':'polygon',
    'eightConnected': False,
    'bestEffort':False,
    'maxPixels':9e10,
    'labelProperty': 'id_dano',
    }
    )



    dNDBR_export=ee.FeatureCollection(dNBR_vector.toList(dNBR_vector.size()))
    #Dissolve the simplified polygons and reduce file size

    geemap.ee_export_vector_to_drive(
        dNDBR_export, description=f"dNBR_2023_postfire_10m_{i}", fileFormat='GeoJSON', folder="dNBR"
    )

134
Exporting dNBR_2023_postfire_10m_134... Please check the Task Manager from the JavaScript Code Editor.
